# Tensorflow

Example code can de found [here](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py) and [here](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/01_Simple_Linear_Model.ipynb)

To play around with a visual neural network see the [TensorFlow Playground](https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.001&regularizationRate=0&noise=0&networkShape=6,2&seed=0.40070&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

mnist

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mnist.data[0].reshape((28,28)))

In [ ]:
mnist.data.shape

In [ ]:
mnist.data.max()

In [ ]:
from sklearn.model_selection import train_test_split

randperm = np.random.permutation(mnist.data.shape[0])

X_full = mnist.data[randperm,:] / 255.0
y_full = [ int(d) for d in mnist.target[randperm] ]

X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=0.33)

In [ ]:
n_hidden_1 = 32 # 1st layer number of neurons
n_hidden_2 = 24 # 2nd layer number of neurons
n_input = X.shape[1] # MNIST data input (img shape: 28*28)
n_classes = np.max(y) + 1 # MNIST total classes (0-9 digits)

learning_rate = 0.001
training_epochs = 100

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, n_input])
y_true_cls = tf.placeholder(tf.int64, [None])
# One hot encoding for Y
y_true = tf.one_hot(y_true_cls, n_classes)


# Create the weights/biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Hidden fully connected layer with 256 neurons
layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])

# Hidden fully connected layer with 256 neurons
layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])

# Output fully connected layer with a neuron for each class
out_layer = tf.matmul(layer_2, weights['out']) + biases['out']


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=out_layer, labels=y_true))

# Try different optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)


init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        
        batch_size = 50
        for i in range(int(X.shape[0]/batch_size)):
          this_X = X[i*batch_size:(i+1)*batch_size,:]
          this_y = y[i*batch_size:(i+1)*batch_size]
          _, c = sess.run([train_op, loss_op], feed_dict={x: this_X, y_true_cls: this_y })
        
        # Display logs per epoch step
        if epoch % 10 == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(c))
    
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(out_layer)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_true, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train Accuracy:", accuracy.eval({ x: X, y_true_cls: y }))
    print("Test Accuracy:", accuracy.eval({ x: X_test, y_true_cls: y_test }))
    
    weights_one = sess.run(weights["h1"])
    plt.imshow(X[0,:].reshape(28,28))
    plt.show()
    
    # To visualise the FIRST neuron in the FIRST layer
    plt.imshow( weights_one[:,0].reshape(28,28) )